code to match input data with student traffic reports for finetuning

In [1]:
import pandas as pd

# Reading multiple sheets from Excel file with input data
sheets_dict = pd.read_excel('./RTVSlo/Podatki - PrometnoPorocilo_2022_2023_2024.xlsx', engine="openpyxl", sheet_name=None)
print(sheets_dict.keys()) #prints keys

dict_keys(['2022', '2023', '2024'])


In [2]:
# Accessing a specific sheet by name
sheet2022 = sheets_dict['2022'] # pandas dataframe
sheet2023 = sheets_dict['2023']
sheet2024 = sheets_dict['2024']

print(sheet2022.columns)

Index(['LegacyId', 'Datum', 'Operater', 'A1', 'B1', 'C1', 'A2', 'B2', 'C2',
       'TitlePomembnoSLO', 'ContentPomembnoSLO', 'TitleNesreceSLO',
       'ContentNesreceSLO', 'TitleZastojiSLO', 'ContentZastojiSLO',
       'TitleVremeSLO', 'ContentVremeSLO', 'TitleOvireSLO', 'ContentOvireSLO',
       'TitleDeloNaCestiSLO', 'ContentDeloNaCestiSLO', 'TitleOpozorilaSLO',
       'ContentOpozorilaSLO', 'TitleMednarodneInformacijeSLO',
       'ContentMednarodneInformacijeSLO', 'TitleSplosnoSLO',
       'ContentSplosnoSLO'],
      dtype='object')


In [3]:
import re

# Function to remove HTML tags and ensure exactly one space between non-space strings separated by tags
def remove_html_tags(text):
    if isinstance(text, str):
        # Replace tags with a single space
        text_no_tags = re.sub(r'<[^>]*>', ' ', text)
        # Collapse multiple spaces into one and strip leading/trailing spaces
        return re.sub(r'\s+', ' ', text_no_tags).strip()
    return text

# Process the data
def process_input_data(data):
    processed_data = []
    for index, row in data.iterrows():
        item = {}
        for column in data.columns:
            value = row[column]
            if pd.notna(value):  # Check if the cell is not empty
                item[column] = remove_html_tags(value)
        processed_data.append(item)

    processed_df = pd.DataFrame(processed_data)
    processed_df = processed_df.reindex(columns=data.columns, fill_value=None) # keep the order of columns
    processed_df = processed_df.dropna(axis=1, how='all')  # Drop columns that are completely empty
    processed_df = processed_df.dropna(axis=0, how='all')  # Drop rows that are completely empty
    
    return processed_df

# Process each sheet
processed_data_2022 = process_input_data(sheet2022)
print(processed_data_2022.columns)
processed_data_2023 = process_input_data(sheet2023)
print(processed_data_2023.columns)
processed_data_2024 = process_input_data(sheet2024)
print(processed_data_2024.columns)

Index(['LegacyId', 'Datum', 'Operater', 'A1', 'B1', 'A2', 'B2', 'C2',
       'TitlePomembnoSLO', 'ContentPomembnoSLO', 'TitleNesreceSLO',
       'ContentNesreceSLO', 'TitleZastojiSLO', 'ContentZastojiSLO',
       'TitleVremeSLO', 'ContentVremeSLO', 'TitleOvireSLO', 'ContentOvireSLO',
       'TitleDeloNaCestiSLO', 'ContentDeloNaCestiSLO', 'TitleOpozorilaSLO',
       'ContentOpozorilaSLO', 'TitleMednarodneInformacijeSLO',
       'ContentMednarodneInformacijeSLO', 'TitleSplosnoSLO',
       'ContentSplosnoSLO'],
      dtype='object')
Index(['LegacyId', 'Datum', 'Operater', 'A1', 'B1', 'B2', 'C2',
       'TitlePomembnoSLO', 'ContentPomembnoSLO', 'TitleNesreceSLO',
       'ContentNesreceSLO', 'TitleZastojiSLO', 'ContentZastojiSLO',
       'TitleVremeSLO', 'ContentVremeSLO', 'TitleOvireSLO', 'ContentOvireSLO',
       'TitleDeloNaCestiSLO', 'ContentDeloNaCestiSLO', 'TitleOpozorilaSLO',
       'ContentOpozorilaSLO', 'TitleMednarodneInformacijeSLO',
       'ContentMednarodneInformacijeSLO', 'Tit

In [4]:
# Save the processed data to CSV files
processed_data_2022.to_csv('./Processed/processed_input_2022.csv', index=False, encoding='utf-8-sig')
processed_data_2023.to_csv('./Processed/processed_input_2023.csv', index=False, encoding='utf-8-sig')
processed_data_2024.to_csv('./Processed/processed_input_2024.csv', index=False, encoding='utf-8-sig')

In [5]:
import pandas as pd
# read from csv output data
rtf_data = pd.read_csv('./Processed/rtf_content_with_category.csv', encoding='utf-8-sig')
print(rtf_data.columns) # Index(['FilePath', 'Datetime', 'Content'], dtype='object')
# convert column Datetime to datetime
rtf_data['Datetime'] = pd.to_datetime(rtf_data['Datetime'], format='%Y-%m-%d %H:%M:%S')
# separate by year
rtf_data_2022 = rtf_data[rtf_data['Datetime'].dt.year == 2022]
rtf_data_2023 = rtf_data[rtf_data['Datetime'].dt.year == 2023]
rtf_data_2024 = rtf_data[rtf_data['Datetime'].dt.year == 2024]
# print the number of rows and first 5 rows
print(rtf_data_2022.shape[0])
print(rtf_data_2022.head())
print(rtf_data_2023.shape[0])
print(rtf_data_2023.head())
print(rtf_data_2024.shape[0])
print(rtf_data_2024.head())

Index(['FilePath', 'Datetime', 'Content', 'TitleCategory'], dtype='object')
8259
                                            FilePath            Datetime  \
0  ./RTVSlo/Podatki - rtvslo.si/Promet 2022/Janua... 2022-01-01 06:00:00   
1  ./RTVSlo/Podatki - rtvslo.si/Promet 2022/Janua... 2022-01-01 06:30:00   
2  ./RTVSlo/Podatki - rtvslo.si/Promet 2022/Janua... 2022-01-01 07:00:00   
3  ./RTVSlo/Podatki - rtvslo.si/Promet 2022/Janua... 2022-01-01 08:00:00   
4  ./RTVSlo/Podatki - rtvslo.si/Promet 2022/Janua... 2022-01-01 08:30:00   

                                             Content         TitleCategory  
0  Prometne informacije       01. 01. 2022  \t   ...  Prometne informacije  
1  Prometne informacije       01. 01. 2022  \t   ...  Prometne informacije  
2  Prometne informacije       01. 01. 2022  \t   ...  Prometne informacije  
3  Prometne informacije       01. 01. 2022  \t   ...  Prometne informacije  
4  Prometne informacije       01. 01. 2022  \t   ...  Prometne informacije  


create json file for each year with following structure:
```
[
    {
        "input": [
            {
                "LegacyId": "...",
                "Datum": "...",
                "Operater": "...",
                "A1": "...",
                "B1": "...",
                "A2": "...",
                "B2": "...",
                "C2": "...",
                "TitlePomembnoSLO": "...",
                "ContentPomembnoSLO": "...",
                "TitleNesreceSLO": "...",
                "ContentNesreceSLO": "...",
                "TitleZastojiSLO": "...",
                "ContentZastojiSLO": "...",
                "TitleVremeSLO": "...",
                "ContentVremeSLO": "...",
                "TitleOvireSLO": "...",
                "ContentOvireSLO": "...",
                "TitleDeloNaCestiSLO": "...",
                "ContentDeloNaCestiSLO": "...",
                "TitleOpozorilaSLO": "...",
                "ContentOpozorilaSLO": "...",
                "TitleMednarodneInformacijeSLO": "...",
                "ContentMednarodneInformacijeSLO": "...",
                "TitleSplosnoSLO": "...",
                "ContentSplosnoSLO": "..."
            }, ...
        ]
        "output": {
            "FilePath": ".\/RTVSlo\/Podatki - rtvslo.si\/Promet ...",
            "Datetime": "...",
            "Content": "...",
            "TitleCategory": "..."
        }
    }
]
```

In [13]:
# create json file for each year with "input" from processed data and "output" from rtf data
# each rtf data row will be matched with the processed data rows from one hour to the time of rtf data
# unless there is no processed data in that time range, then we increase the time range to 2 hours
def create_json_file(year, rtf_data, processed_data, time_section_in_hours):
    json_data = []
    processed_data['Datum'] = pd.to_datetime(processed_data['Datum'], format='%Y-%m-%d %H:%M:%S')
    rtf_data['Datetime'] = pd.to_datetime(rtf_data['Datetime'], format='%Y-%m-%d %H:%M:%S')
    for index_rtf, row_rtf in rtf_data.iterrows():
        item = {}
        row_rtf_copy = row_rtf.copy()
        # change Datetime to string to put in json
        row_rtf_copy['Datetime'] = row_rtf_copy['Datetime'].strftime('%Y-%m-%d %H:%M:%S')
        item['output'] = row_rtf_copy.to_dict()
        item['input'] = []
        # get the datetime of the rtf data
        datetime = row_rtf['Datetime']
        # get the processed data rows that are within one hour of the rtf data
        start_time = datetime - pd.Timedelta(hours=time_section_in_hours)
        end_time = datetime
        # change Datum to datetime
        
        filtered_data = processed_data[(processed_data['Datum'] >= start_time) & (processed_data['Datum'] <= end_time)]
        # if there are no rows in the time range, increase the time range to 2 hours
        if filtered_data.shape[0] == 0:
            start_time = datetime - pd.Timedelta(hours=time_section_in_hours * 2)
            end_time = datetime
            filtered_data = processed_data[(processed_data['Datum'] >= start_time) & (processed_data['Datum'] <= end_time)]
        for index, row in filtered_data.iterrows():
            # change Datetime and Datum to string
            row_copy = row.copy()
            row_copy['Datum'] = pd.to_datetime(row_copy['Datum'], format='%Y-%m-%d %H:%M:%S')
            row_copy['Datum'] = row_copy['Datum'].strftime('%Y-%m-%d %H:%M:%S')
            # add only non-empty values to the json
            row_dict = row_copy.to_dict()
            row_dict = {k: v for k, v in row_dict.items() if pd.notna(v)}
            item['input'].append(row_dict)
        json_data.append(item)
    
    return json_data

In [3]:
# create json file and save it
import json
def save_json_file(name, json_data):
    with open(f'./Processed/{name}.json', 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

In [14]:
# create json file for each year
json_data_2022 = create_json_file(2022, rtf_data_2022, processed_data_2022, 1)
save_json_file("input_output_all_data_2022", json_data_2022)
json_data_2023 = create_json_file(2023, rtf_data_2023, processed_data_2023, 1)
save_json_file("input_output_all_data_2023", json_data_2023)
json_data_2024 = create_json_file(2024, rtf_data_2024, processed_data_2024, 1)
save_json_file("input_output_all_data_2024", json_data_2024)

/tmp/ipykernel_122489/3132823273.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rtf_data['Datetime'] = pd.to_datetime(rtf_data['Datetime'], format='%Y-%m-%d %H:%M:%S')
/tmp/ipykernel_122489/3132823273.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rtf_data['Datetime'] = pd.to_datetime(rtf_data['Datetime'], format='%Y-%m-%d %H:%M:%S')
/tmp/ipykernel_122489/3132823273.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

# reducing data
ok time to reduce the data that was put in those input-output json files because we probably don't need them all

i think we either remove input news datetime or the ID

and who wrote the news is also probably not important that much

i will also try to remove the duplicate fields among the inputs because same news can appear multiple times. 
This might make analyzing if news matches easier. 

In [21]:
def remove_duplicate_fields_in_inputs(json_data):
    all_pair_num = 0
    no_input_num = 0
    for item in json_data:
        input_list = item['input']
        if not input_list or not isinstance(input_list, list):
            # print("No input data found or input is not a list------>")
            # print(input_list)
            # print(item['output'])
            no_input_num += 1
            continue
        all_pair_num += 1
        # Always remove 'LegacyId' and 'Operater'
        for row in input_list:
            row.pop('LegacyId', None)
            row.pop('Operater', None)
        # Collect all fields present in any dict (except those two)
        all_fields = set()
        for row in input_list:
            all_fields.update(row.keys())
        all_fields.discard('LegacyId')
        all_fields.discard('Operater')
        # For each field, track all (value, index) pairs
        for field in all_fields:
            value_indices = {}
            for idx, row in enumerate(input_list): # idx is the index of the dictionary, row is the dictionary
                value = row.get(field)
                if value is not None:
                    value_indices.setdefault(value, []).append(idx)
            # Remove the key from all but the last occurrence
            for value, indices in value_indices.items():
                for idx in indices[:-1]:
                    if field in input_list[idx] and input_list[idx][field] == value:
                        del input_list[idx][field]
    # Remove empty dictionaries or those that have only 'Datum' field from input_list
    input_list[:] = [row for row in input_list if len(row) > 1]
    print("no input num: ", no_input_num)
    print("all pair num: ", all_pair_num)
    return json_data

In [16]:
test = {
        "output": {
            "FilePath": "./RTVSlo/Podatki - rtvslo.si/Promet 2022/Januar 2022/TMP-808.rtf",
            "Datetime": "2022-01-01 06:00:00",
            "Content": "Prometne informacije       01. 01. 2022  \t   06.00           1. in 2. program \n\nPodatki o prometu.\n\nPonekod po državi megla zmanjšuje vidljivost.\n\nZaradi del je na vzhodni mariborski obvoznici v obe smeri zaprt prehitevalni pas med razcepom Dragučova in priključkom Pesnica.\n"
        },
        "input": [
            {
                "LegacyId": 1283527,
                "Datum": "2022-01-01 05:15:40",
                "Operater": "DARS Tjaša Krmac, GNC",
                "B1": "Vreme Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost! Omejitve za tovorna vozila Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton: - danes, 1. 1., od 8. do 22. ure; - v nedeljo, 2. 1., od 8. do 22. ure. Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več. Dela Na primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.",
                "ContentVremeSLO": "Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!",
                "TitleDeloNaCestiSLO": "Dela",
                "ContentDeloNaCestiSLO": "Na primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.",
                "TitleOpozorilaSLO": "Omejitve za tovorna vozila",
                "ContentOpozorilaSLO": "Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton: - danes, 1. 1., od 8. do 22. ure; - v nedeljo, 2. 1., od 8. do 22. ure. Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več."
            },
            {
                "LegacyId": 1283528,
                "Datum": "2022-01-01 05:33:28",
                "Operater": "DARS Barbara Janežič, PIC",
                "B1": "Vreme Ponekod po Sloveniji megla zmanjšuje vidljivost. Prilagodite hitrost! Omejitve za tovorna vozila Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton: - danes, 1. 1., od 8. do 22. ure; - v nedeljo, 2. 1., od 8. do 22. ure. Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več. Dela Na primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.",
                "ContentVremeSLO": "Ponekod po Sloveniji megla zmanjšuje vidljivost. Prilagodite hitrost!",
                "TitleDeloNaCestiSLO": "Dela",
                "ContentDeloNaCestiSLO": "Na primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.",
                "TitleOpozorilaSLO": "Omejitve za tovorna vozila",
                "ContentOpozorilaSLO": "Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton: - danes, 1. 1., od 8. do 22. ure; - v nedeljo, 2. 1., od 8. do 22. ure. Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več."
            },
            {
                "LegacyId": 1283529,
                "Datum": "2022-01-01 05:34:08",
                "Operater": "DARS Tjaša Krmac, GNC",
                "B1": "Vreme Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost! Omejitve za tovorna vozila Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton: - danes, 1. 1., od 8. do 22. ure; - v nedeljo, 2. 1., od 8. do 22. ure. Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več. Dela Na primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.",
                "ContentVremeSLO": "Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!",
                "TitleDeloNaCestiSLO": "Dela",
                "ContentDeloNaCestiSLO": "Na primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.",
                "TitleOpozorilaSLO": "Omejitve za tovorna vozila",
                "ContentOpozorilaSLO": "Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton: - danes, 1. 1., od 8. do 22. ure; - v nedeljo, 2. 1., od 8. do 22. ure. Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več."
            },
            {
                "LegacyId": 1283530,
                "Datum": "2022-01-01 05:35:57",
                "Operater": "DARS Barbara Janežič, PIC",
                "B1": "Vreme Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost! Omejitve za tovorna vozila Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton: - danes, od 8. do 22. ure; - v nedeljo, 2. 1., od 8. do 22. ure. Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več. Dela Na primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.",
                "ContentVremeSLO": "Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!",
                "TitleDeloNaCestiSLO": "Dela",
                "ContentDeloNaCestiSLO": "Na primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.",
                "TitleOpozorilaSLO": "Omejitve za tovorna vozila",
                "ContentOpozorilaSLO": "Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton: - danes, od 8. do 22. ure; - v nedeljo, 2. 1., od 8. do 22. ure. Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več."
            }
        ]
    }

test_no_duplicates = remove_duplicate_fields_in_inputs([test])
import json

print(json.dumps(test_no_duplicates[0]['input'], ensure_ascii=False, indent=4))

[
    {
        "Datum": "2022-01-01 05:33:28",
        "B1": "Vreme Ponekod po Sloveniji megla zmanjšuje vidljivost. Prilagodite hitrost! Omejitve za tovorna vozila Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton: - danes, 1. 1., od 8. do 22. ure; - v nedeljo, 2. 1., od 8. do 22. ure. Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. Več. Dela Na primorski avtocesti je ponovno odprt priključek Črni Kal v obe smeri.",
        "ContentVremeSLO": "Ponekod po Sloveniji megla zmanjšuje vidljivost. Prilagodite hitrost!"
    },
    {
        "Datum": "2022-01-01 05:34:08",
        "B1": "Vreme Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost! Omejitve za tovorna vozila Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton: - danes, 1. 1., od 8. do 22. ure; - v nedeljo, 2. 1., od 8. do 22. ure. Od 30. decembra je v veljavi sprememba omejitve za

In [22]:
# read json files
import json

def read_json_file(year):
    with open(f'./Processed/input_output_all_data_{year}.json', 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    return json_data

json_data_2022 = read_json_file(2022)
json_data_2023 = read_json_file(2023)
json_data_2024 = read_json_file(2024)
# remove duplicates
json_data_2022_reduced = remove_duplicate_fields_in_inputs(json_data_2022)
json_data_2023_reduced = remove_duplicate_fields_in_inputs(json_data_2023)
json_data_2024_reduced = remove_duplicate_fields_in_inputs(json_data_2024)

no input num:  22
all pair num:  8237
no input num:  7
all pair num:  8997
no input num:  10
all pair num:  2608


In [23]:
# save json files
save_json_file("input_output_all_data_2022_reduced", json_data_2022)
save_json_file("input_output_all_data_2023_reduced", json_data_2023)
save_json_file("input_output_all_data_2024_reduced", json_data_2024)

In [ ]:
"""
categories = [
    "Prometne informacije",
    "Nujna prometna informacija",
    "NOVA Prometna informacija",
    "Podatki o prometu",
    "NOVA nujna prometna informacija"
]
"""
def only_nujna_porocila(year):
    with open(f'./Processed/input_output_all_data_{year}_reduced.json', 'r', encoding='utf-8') as f:
        json_data = json.load(f)
    
    filtered_data = []
    for item in json_data:
        if item['output']['TitleCategory'] in ["Nujna prometna informacija", "NOVA nujna prometna informacija"]:
            # Check if 'input' is a list and has more than one entry
            if isinstance(item['input'], list) and len(item['input']) > 1:
                filtered_data.append(item)
    
    return filtered_data

# Filter the data for each year
filtered_data_2022 = only_nujna_porocila(2022)
filtered_data_2023 = only_nujna_porocila(2023)
filtered_data_2024 = only_nujna_porocila(2024)
# Save the filtered data to new JSON files
save_json_file("nujna_porocila_2022", filtered_data_2022)
save_json_file("nujna_porocila_2023", filtered_data_2023)
save_json_file("nujna_porocila_2024", filtered_data_2024)
# Print the number of entries in each filtered dataset
print(f"Filtered Nujna Porocila 2022: {len(filtered_data_2022)} entries")
print(f"Filtered Nujna Porocila 2023: {len(filtered_data_2023)} entries")
print(f"Filtered Nujna Porocila 2024: {len(filtered_data_2024)} entries")

Filtered Nujna Porocila 2022: 31 entries
Filtered Nujna Porocila 2023: 93 entries
Filtered Nujna Porocila 2024: 41 entries


In [5]:
# merge all filtered data into one JSON file
def merge_filtered_data(*filtered_data_list):
    merged_data = []
    for filtered_data in filtered_data_list:
        merged_data.extend(filtered_data)
    return merged_data
# Merge the filtered data for all years
merged_filtered_data = merge_filtered_data(filtered_data_2022, filtered_data_2023, filtered_data_2024)
# Save the merged filtered data to a new JSON file
save_json_file("nujna_porocila_all_years", merged_filtered_data)
# Print the number of entries in the merged dataset
print(f"Merged Nujna Porocila All Years: {len(merged_filtered_data)} entries")

Merged Nujna Porocila All Years: 165 entries


In [9]:
def nujna_normal_next_prev():
    with open(f'./Processed/input_output_all_data_2022_reduced.json', 'r', encoding='utf-8') as f:
        json_data2022 = json.load(f)
    f.close()
    with open(f'./Processed/input_output_all_data_2023_reduced.json', 'r', encoding='utf-8') as f:
        json_data2023 = json.load(f)
    f.close()
    with open(f'./Processed/input_output_all_data_2024_reduced.json', 'r', encoding='utf-8') as f:
        json_data2024 = json.load(f)
    f.close()
    # Merge the data for all years
    merged_data = json_data2022 + json_data2023 + json_data2024
    # Filter the merged data for 'Nujna prometna informacija' and 'NOVA nujna prometna informacija' and one previous and one next
    filtered_data = []
    for i, item in enumerate(merged_data):
        if item['output']['TitleCategory'] in ["Nujna prometna informacija", "NOVA nujna prometna informacija"]:
            data_triplet = []
            # Add the previous item if it exists
            # Check if 'input' is a list
            # remove from input each dict that is empty or has only 'Datum' field
            if i > 0 and isinstance(merged_data[i - 1]['input'], list) and len(item['input']) > 0:
                input_prev = merged_data[i - 1]['input']
                input_prev = [d for d in input_prev if len(d) > 1]
                merged_data[i - 1]['input'] = input_prev
                # If the previous item has input, add it to the triplet
                if len(merged_data[i - 1]) > 0 and len(merged_data[i - 1]['input']) > 0:
                    data_triplet.append(merged_data[i - 1])
            # Check if the current item has input
            # remove from input each dict that is empty or has only 'Datum' field
            if len(item['input']) > 0:
                input_current = item['input']
                input_current = [d for d in input_current if len(d) > 1]
                item['input'] = input_current
                # If the current item has input, add it to the triplet
                if len(item) > 0 and len(item['input']) > 0:
                    data_triplet.append(item)
            # Add the next item if it exists
            # remove from input each dict that is empty or has only 'Datum' field
            if i < len(merged_data) - 1 and isinstance(merged_data[i + 1]['input'], list) and len(item['input']) > 0:
                input_next = merged_data[i + 1]['input']
                input_next = [d for d in input_next if len(d) > 1]
                merged_data[i + 1]['input'] = input_next
                # If the next item has input, add it to the triplet
                if len(merged_data[i + 1]) > 0 and len(merged_data[i + 1]['input']) > 0:
                    data_triplet.append(merged_data[i + 1])
            # If the triplet has at least one item, add it to the filtered data
            if(len(data_triplet) > 0):
                filtered_data.append(data_triplet)
    return filtered_data

# Get the filtered data with previous and next items
filtered_data_with_prev_next = nujna_normal_next_prev()
# Save the filtered data with previous and next items to a new JSON file
save_json_file("nujna_normal_prev_next", filtered_data_with_prev_next)
# Print the number of entries in the filtered dataset with previous and next items
print(f"Filtered Nujna Normal Prev Next: {len(filtered_data_with_prev_next)} entries")
#print the first entry in the filtered dataset with previous and next items
print(json.dumps(filtered_data_with_prev_next[0], ensure_ascii=False, indent=4))
    

Filtered Nujna Normal Prev Next: 171 entries
[
    {
        "output": {
            "FilePath": "./RTVSlo/Podatki - rtvslo.si/Promet 2022/Januar 2022/TMP-417.rtf",
            "Datetime": "2022-01-16 14:30:00",
            "Content": "Prometne informacije       16. 01. 2022       14.30       2. program \n\nPodatki o prometu.\n\nPonekod po državi megla zmanjšuje vidljivost.\n\nZaradi del je na pomurski avtocesti proti Mariboru zaprt vozni pas med priključkoma Sveta Trojica in Lenart.\n\nNa mejnem prehodu Obrežje vozniki osebnih vozil in avtobusov na vstop v državo čakajo do eno uro, v Gruškovju pa je povečan promet osebnih vozil pri vstopu v državo.\n",
            "TitleCategory": "Prometne informacije"
        },
        "input": [
            {
                "Datum": "2022-01-16 13:52:15",
                "A1": "Pozor! Na štajerski avtocesti zaprt predor Jasovnik proti Ljubljani zaradi okvare vozila.",
                "B1": "Nesreče Na ljubljanski severni obvoznici na izvozu Bežig